In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from datasets import load_dataset
from tqdm.notebook import tqdm

from navec import Navec

import matplotlib.pyplot as plt
import numpy as np

In [2]:
from extractor.model import Extractor
from extractor.utils import create_target_mask, TaggingDataset

In [3]:
dataset = load_dataset('IlyaGusev/gazeta', revision="v2.0")

No config specified, defaulting to: gazeta/default
Found cached dataset gazeta (/home/goncharovglebig/.cache/huggingface/datasets/IlyaGusev___gazeta/default/2.0.0/c329f0fc1c22ab6e43e0045ee659d0d43c647492baa2a6ab3a5ea7dac98cd552)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train_dataset = TaggingDataset(dataset['train']['text'], dataset['train']['summary'])
val_dataset = TaggingDataset(dataset['validation']['text'], dataset['validation']['summary'])

  0%|          | 0/60964 [00:00<?, ?it/s]

  0%|          | 0/6369 [00:00<?, ?it/s]

In [5]:
navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')
navec_emb_size = 300
vocab = navec.vocab

In [6]:
def collate_batch(batch):
    art_list, target_list = [], []
    for art, target in batch:
        art_encoded = [vocab[tok] if tok in vocab else vocab.unk_id
                       for tok in art]
        art_list.append(torch.tensor(np.array(art_encoded)))
        target_list.append(torch.FloatTensor(np.array(target)))
        
    art_tensor_padded = pad_sequence(art_list, padding_value=vocab.pad_id).T
    target_tensor_padded = pad_sequence(target_list, padding_value=0).T
    return art_tensor_padded.to(device), target_tensor_padded.to(device)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=256, collate_fn=collate_batch)
val_loader = DataLoader(val_dataset, batch_size=256, collate_fn=collate_batch)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = Extractor()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
min_loss = 99999
model.to(device)

params = list(model.parameters())
optimizer = torch.optim.Adagrad(params, lr=0.1, initial_accumulator_value=0.1)
criternion = nn.BCEWithLogitsLoss()

epoch_num = 45
train_loss_list, val_loss_list = [], []

for ep in tqdm(range(epoch_num)):
    for train_batch in tqdm(train_loader):
        src, target = train_batch
        out = model(src).squeeze()
        
        loss = criternion(out, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss = loss.detach().item()
        train_loss_list.append(train_loss)

    #Validation loop
    val_batch_loss = []
    with torch.no_grad():
        for val_batch in val_loader:
            src, target = train_batch
            out = model(src).squeeze()
            loss = criternion(out, target)
            val_batch_loss.append(loss.detach().item())
    val_loss = np.mean(val_batch_loss)
    val_loss_list.append(val_loss)
    if val_loss < min_loss:
        min_loss = val_loss
        print('Saving best model')
        torch.save(model, f'extractor.pth')

    print(f'For epoch #{ep} train loss {np.mean(train_loss_list[-250_000:])}, val loss {val_loss}')
    

/home/goncharovglebig/project/summ_venv/lib/python3.7/site-packages/slovnet/model/emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/239 [00:00<?, ?it/s]

Saving best model
For epoch #0 train loss 0.5898529271201609, val loss 0.5955839490890503


  0%|          | 0/239 [00:00<?, ?it/s]

...

  0%|          | 0/239 [00:00<?, ?it/s]

Saving best model
For epoch #32 train loss 0.4140474852257179, val loss 0.4110026872158051


  0%|          | 0/239 [00:00<?, ?it/s]

For epoch #33 train loss 0.4133167070669697, val loss 0.4113087594509125


  0%|          | 0/239 [00:00<?, ?it/s]

In [ ]:
plt.plot(train_loss_list, label='train');
plt.legend();

In [ ]:
plt.plot(val_loss_list, label='val');
plt.legend();